# My first pairwise BLASTn

The NCBI C++ Toolkit is a rich library that contains a comprehensive data model and interfaces to several sequence analysis methods. The most important one is [BLAST](https://en.wikipedia.org/wiki/BLAST_(biotechnology)), the Basic Local Alignment Search Tool, which allows retrieving sequences similar to a query in a database of sequences, among other things. The PyNCBItk library implements a high-level interface to the BLAST methods (such as `blastn`, `blastp`, etc.) in the `pyncbitk.algo.blast` module. 

In [ ]:
import pyncbitk
pyncbitk.__version__

One of the easiest application of BLAST is running a pairwise nucleotide BLAST. It takes a query sequence, a subject sequence, and returns the local alignments between these two sequences. Being one of the most common analyses, it should not be too hard to set-up, right? 

## Getting sequence data

Let's start by downloading some data, for instance two related genomes of *Escherichia coli*, of strains K12 and K.

In [ ]:
import urllib.request
import shutil
import pathlib

genomes = {
    "LN832404": 802133627,  # Escherichia coli K-12
    "AE014075": 26111730,   # Escherichia coli O157
}

for accession, id_ in genomes.items():
    with urllib.request.urlopen(f"https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?save=file&db=nuccore&report=fasta&id={id_}") as res:
        with pathlib.Path("data").joinpath(f"{accession}.fna").open("wb") as dst:
            shutil.copyfileobj(res, dst)

Now that we have two FASTA files, we should be able to run our `blastn` query. With the command line, we'd run something like:
```console
$ blastn -query data/LN832404.fna -subject data/AE014075.fna
```

However, with the NCBI C++ Toolkit and PyNCBItk, we cannot simply use filenames: we need to load data first. 

## Loading data with the NCBI Toolkit

Since our sequences are in FASTA format, we can use the `FastaReader` class from the `pyncbitk.objtools` module to load the sequences into Python objects. Let's use the `FastaReader`, and ignore the `split=False` argument for now (don't worry, I will explain later).

In [ ]:
from pyncbitk.objtools import FastaReader

k12 = FastaReader(pathlib.Path("data").joinpath("LN832404.fna"), split=False).read()
o157 = FastaReader(pathlib.Path("data").joinpath("AE014075.fna"), split=False).read()

The two objects we just loaded are instances of the `BioSeq` class. They store the identifier(s) and the sequence data for each of these two sequences. 

## Setting up the BLAST query

Now that the data has been loaded, we can prepare a BLASTn runner, and configure it with the same configuration values as the command line:

In [ ]:
from pyncbitk.algo.blast import BlastN
blastn = BlastN(evalue=1e-5)

## Running the BLAST query

Our BLASTn runner is now configured! All we have to do is run the search: for this, we can use the `run` method common to all `Blast` runner objects. 

<!-- using a dedicated object implementing [Command pattern](https://en.wikipedia.org/wiki/Command_pattern) allows reusing a BLASTn configuration for different query/subject pairs, which ensure they are always using the same parameters. In particular, `BlastN` instances are thread-safe, and the `BlastN.run -->

In [ ]:
results = blastn.run(k12, o157)

Great! Our search succeeded. However... What exactly is in these `results`? The BLAST binaries in the command line let us configure the output, or redirects the alignments to the standard output. Here, none of that happened. So what exactly are those results?

In [ ]:
type(results)

## Analyzing the BLAST results

`Blast.run` always returns a `SearchResultSet`, which is a list of `SearchResults` objects, with at most one for each query. Since we only had one query, we can just use the first (and only) element.

In [ ]:
result = results[0]
type(result)

We now have a single `SearchResults`, which contains the results for our query. We can check the identifier of the query with the `query_id` attribute, and the alignments with the `alignments` attribute:

In [ ]:
print("Query:", result.query_id)
print("Alignments:", len(result.alignments))

Each `SeqAlign` objects in the alignments have various attributes that can be used to display the identifiers of the query and target sequences, the bit-score and E-value of the BLAST alignment:

In [ ]:
from itertools import islice
for alignment in islice(result.alignments, 10):  # show the first 10 alignments
    print(alignment[0].id, alignment[1].id, alignment.evalue, alignment.bitscore, alignment.matches, alignment.percent_identity, alignment.alignment_length, sep="\t")

Note that we didn't get the respective coordinates in the query and subject sequences. That's because the alignment format of the core object model (the `SeqAlign` class) actually stores the data in a compact form, which makes it more complicated to extract coordinates. Luckily, a dedicated class for that is available in the `objtools` module, the `AlignMap`:

In [ ]:
from itertools import islice
from pyncbitk.objtools import AlignMap
for alignment in islice(result.alignments, 5):  # show the first 5 alignments
    alimap = AlignMap(alignment.segments)
    print(alignment[0].id, alignment[1].id, alimap[0].sequence_start, alimap[0].sequence_stop, alimap[1].sequence_start, alimap[1].sequence_stop, sep="\t")

Good job, you just ran your first pairwise BLAST!